In [ ]:
!pip install -U sagemaker --quiet # Ensure latest version of SageMaker is installed

In [ ]:
import boto3
import sagemaker
import time

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()
s3_client = boto3.client("sagemaker")

model_package_group_name = "deepfake-detection-models"  
pipeline_name = "Deepfake-detection-pipeline"  

## Get the pipeline instance

In [ ]:
from pipelines.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name
)

## Submit the pipeline to SageMaker and start execution

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

In [ ]:
execution.describe()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

## Deploy the Model Version to Production

In [ ]:
from botocore.exceptions import ClientError

def get_approved_package(sm_client, model_package_group_name):
    """Gets the latest approved model package for a model package group.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        The SageMaker Model Package ARN.
    """
    try:
        # Get the latest approved model package
        response = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # Fetch more packages if none returned with continuation token
        while len(approved_packages) == 0 and "NextToken" in response:
            print("Getting more packages for token: {}".format(response["NextToken"]))
            response = sm_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        # Return error if no packages found
        if len(approved_packages) == 0:
            error_message = (
                f"No approved ModelPackage found for ModelPackageGroup: {model_package_group_name}"
            )
            print(error_message)
            raise Exception(error_message)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        print(f"Identified the latest approved model package: {model_package_arn}")
        return model_package_arn
    except ClientError as e:
        error_message = e.response["Error"]["Message"]
        print(error_message)
        raise Exception(error_message)

In [ ]:
def retrieve_data_model_from_latest_approved_model(sm_client, model_package_group_name):
    """Retrieve data and model from the latest approved model.

    Args:
        model_package_group_name: The model package group name.

    Returns:
        Model url and output artifacts
    """
    # Get the latest approved package
    model_package_arn = get_approved_package(sm_client, model_package_group_name)
    
    ## Get model url
    model_url = sm_client.describe_model_package(
        ModelPackageName=model_package_arn
    )['InferenceSpecification']['Containers'][0]['ModelDataUrl']
    
    ## Get the trial name
    trial_arn = sm_client.describe_model_package(
            ModelPackageName=model_package_arn
        )['MetadataProperties']['GeneratedBy']
    position = trial_arn.rfind('/')
    trial_name = trial_arn[position+1:]
    
    # Get preprocess trial component
    preprocess_component = sm_client.list_trial_components(
            TrialName=trial_name,
        )['TrialComponentSummaries'][-1]['TrialComponentName']
    
    # Get output artifacts
    component_trial_describe = sm_client.describe_trial_component(
            TrialComponentName=preprocess_component
        )
    output_artifacts = component_trial_describe['OutputArtifacts']
    
    return model_url, output_artifacts

In [ ]:
deepfake_model_url, deepfake_output_artifacts = retrieve_data_model_from_latest_approved_model(s3_client, model_package_group_name)

## Clean up (optional)
Delete the model registry and the pipeline to keep the studio environment tidy.

In [ ]:
def delete_model_package_group(sm_client, package_group_name):
    try:
        model_versions = sm_client.list_model_packages(ModelPackageGroupName=package_group_name)

    except Exception as e:
        print("{} \n".format(e))
        return

    for model_version in model_versions["ModelPackageSummaryList"]:
        try:
            sm_client.delete_model_package(ModelPackageName=model_version["ModelPackageArn"])
        except Exception as e:
            print("{} \n".format(e))
        time.sleep(0.5)  # Ensure requests aren't throttled

    try:
        sm_client.delete_model_package_group(ModelPackageGroupName=package_group_name)
        print("{} model package group deleted".format(package_group_name))
    except Exception as e:
        print("{} \n".format(e))
    return


def delete_sagemaker_pipeline(sm_client, pipeline_name):
    try:
        sm_client.delete_pipeline(
            PipelineName=pipeline_name,
        )
        print("{} pipeline deleted".format(pipeline_name))
    except Exception as e:
        print("{} \n".format(e))
        return

In [ ]:
delete_model_package_group(s3_client, model_package_group_name)
delete_sagemaker_pipeline(s3_client, pipeline_name)